In [ ]:
#작업을 진행하기 전에, DB이름 : registration으로 먼저 만들어주고 DB 유저를 생성해주세요
#1. Student 테이블 만들기
#mysql에 접속이 안되면 본인 로컬 mysql에 계정 권한을 주고 다시 시도해주세요
import scipy.io
import csv
import pymysql

db_user = 'jhyun1' #접속할 DB 이름
db_host = 'localhost' #접속할 DB 호스트
db_password = 's852130'#DB 비밀번호
db_charset = 'utf8'
db_name = 'registration'

csv_path = './csv/'

conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
curs = conn.cursor()

CREATE_STUDENT_TABLE = "create table student("\
"student_id varchar(12) NOT NULL UNIQUE KEY,"\
"password varchar(45) NOT NULL,"\
"name varchar(45) NOT NULL,"\
"department varchar(45) NOT NULL,"\
"term varchar(45) DEFAULT NULL,"\
"division varchar(45) DEFAULT NULL,"\
"phone_number varchar(20) DEFAULT NULL,"\
"PRIMARY KEY(student_id)"\
") ENGINE = InnoDB;"
print(CREATE_STUDENT_TABLE)

AffectedRows = curs.execute(CREATE_STUDENT_TABLE)
conn.commit()
print('Number of affected rows: %d' %(AffectedRows))
conn.close()

In [ ]:
#2. 학정정보.csv에서 데이터를 읽어와 테이블에 넣기
conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
curs = conn.cursor()
with open(csv_path + '학정 정보.csv', 'rt', encoding='UTF8') as f:
    csvReader = csv.reader(f)
    i = 0
    for row in csvReader:
        if i == 0: #첫 행은 거름
            i += 1
            continue
        student_id = row[0] #학번
        password = 'q1w2e3r4' #비밀번호 값은 임의로 넣음
        name = row[1]    #이름
        department = row[2] #학과
        term = row[3] if len(row[3]) != 0 else None #학적사항
        division = row[4] if len(row[4]) != 0 else None #분반
        phone_number = row[5] if len(row[5]) != 0 else None #전화번호
        sql = """insert into student(student_id, password, name, department, term, division, phone_number) """\
        """values(%s, %s, %s, %s, %s, %s, %s)"""
        curs.execute(sql, (student_id, password, name, department, term, division, phone_number))
    
    conn.commit()
    conn.close()

In [ ]:
#3. 개설교과목 정보 테이블 선언 및 데이터 추가
CREATE_SUBJECT_TABLE = "CREATE TABLE subject("\
"subject_id varchar(100) not null,"\
"term varchar(10) not null,"\
"subject_name varchar(100) not null,"\
"division varchar(100) DEFAULT NULL,"\
"is_major tinyint(1) NOT NULL,"\
"class_hours INT NOT NULL,"\
"professor_name varchar(100) DEFAULT NULL,"\
"day1 varchar(10) DEFAULT NULL,"\
"time1 varchar(10) DEFAULT NULL,"\
"day2 varchar(10) DEFAULT NULL,"\
"time2 varchar(10) DEFAULT NULL,"\
"day3 varchar(10) DEFAULT NULL,"\
"time3 varchar(10) DEFAULT NULL,"\
"people INT,"\
"prerequisite varchar(100) DEFAULT NULL,"\
"PRIMARY KEY(subject_id, term)"\
") ENGINE = InnoDB;"
print(CREATE_SUBJECT_TABLE)
try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SUBJECT_TABLE)
    conn.commit()
    
    with open(csv_path + '개설 교과목 정보.csv', 'rt', encoding='UTF8') as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into subject(subject_id, term, subject_name, division, is_major, class_hours, professor_name, day1, time1, day2, time2, day3, time3, people, prerequisite)"""\
            """values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
            subject_id = row[0] #과목 코드
            term = row[1] #학기
            subject_name = row[2] #과목 이름
            division = row[3] if len(row[3]) != 0 else None #분반
            is_major = 1 if row[4] == '전공' else 0 #전공 여부
            class_hours = row[5] #수업시수
            professor_name = row[6] if len(row[6]) != 0 else None #교수명
            day1 = row[7] if len(row[7]) != 0 else None #요일 1
            time1 = row[8] if len(row[8]) != 0 else None #시간 1
            day2 = row[9] if len(row[9]) != 0 else None #요일2
            time2 = row[10] if len(row[10]) != 0 else None #시간 2
            day3 = row[11] if len(row[11]) != 0 else None #요일 3
            time3 = row[12] if len(row[12]) != 0 else None #시간 3
            people = row[13] #인원수
            prerequisite = row[14] if len(row[14]) != 0 else None #선수과목
            curs.execute(sql, (subject_id, term, subject_name, division, is_major, class_hours, professor_name, day1, time1, day2, time2, day3, time3, people, prerequisite))
        conn.commit()
finally:
    conn.close()

In [ ]:
#4. 강의평가 테이블 선언 및 데이터 추가
CREATE_SUBJECT_EVAL_TABLE = "CREATE TABLE subject_eval("\
"subject_id VARCHAR(100) NOT NULL,"\
"term VARCHAR(10) NOT NULL,"\
"subject_eval varchar(2000) NOT NULL,"\
"CONSTRAINT FK_ID_TERM FOREIGN KEY(subject_id, term) REFERENCES subject(subject_id, term) ON UPDATE CASCADE"\
") ENGINE = InnoDB;"
print(CREATE_SUBJECT_EVAL_TABLE)
try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SUBJECT_EVAL_TABLE)
    conn.commit()
    
    with open(csv_path + "강의평가.csv", "rt", encoding="UTF8") as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into subject_eval(subject_id, term, subject_eval) values(%s, %s, %s)"""
            subject_id = row[0]
            term = row[1]
            subject_eval = row[2]
            curs.execute(sql, (subject_id, term, subject_eval))
    conn.commit()
finally:
    conn.close()

In [ ]:
#5. 선수과목 테이블 선언 및 데이터 추가
CREATE_SUBJECT_PREREQUISITE_TABLE = "CREATE TABLE subject_prerequisite("\
"subject_name varchar(100) NOT NULL,"\
"prerequisite varchar(100) NOT NULL"\
");"
try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SUBJECT_PREREQUISITE_TABLE)
    conn.commit()
    
    with open(csv_path + '선수과목정보.csv', "rt", encoding="UTF8") as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into subject_prerequisite values(%s, %s)"""
            subject_name = row[0] #과목명
            prerequisite = row[1] #선수과목
            curs.execute(sql, (subject_name, prerequisite))
    conn.commit()
finally:
    conn.close()

In [ ]:
#6. 수강내역 테이블 선언 및 추가
CREATE_STUDENT_GRADE_TABLE = "CREATE TABLE attend("\
"student_id varchar(12) NOT NULL,"\
"subject_id varchar(100) NOT NULL,"\
"term varchar(10) NOT NULL,"\
"grade varchar(10)"\
");"

try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_STUDENT_GRADE_TABLE)
    conn.commit()
    
    with open(csv_path + '수강내역.csv', "rt", encoding="UTF8") as f:
        csvReader = csv.reader(f)
        i = 0
        for row in csvReader:
            if i == 0:
                i += 1
                continue
            sql = """insert into student_grade values(%s, %s, %s, %s)"""
            student_id = row[0] #학번
            subject_id = row[1] #학정번호
            term = row[2] #수강학기
            grade = row[3] if len(row[3]) != 0 else None #성적
            curs.execute(sql, (student_id, subject_id, term, grade))
    conn.commit()
finally:
    conn.close()

In [ ]:
#7. 성적 종합 정보
CREATE_SCORE_TABLE = "CREATE TABLE score("\
"score_id int AUTO_INCREMENT,"\
"student_id varchar(12) NOT NULL,"\
"term varchar(10),"\
"grade int,"\
"avg_grade double,"\
"CONSTRAINT FK_student_id FOREIGN KEY(student_id) REFERENCES student(student_id) ON UPDATE CASCADE,"\
"PRIMARY KEY(score_id)"\
");"

try:
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name, charset=db_charset)
    curs = conn.cursor()
    curs.execute(CREATE_SCORE_TABLE)
    conn.commit()
    
finally:
    conn.close()